In [31]:
#Importamos las librerias necesarias para comenzar con nuestro modelo de recomendacion
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
import nltk as nlt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string 


In [32]:
#Descargamos las stopwords
nlt.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/pabloclementi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/pabloclementi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [33]:
#Leemos nuestro archivo parquet. 
df_recomendacion = pd.read_parquet('data/df_recomendacion.parquet')
# df_recomendacion.head() 


In [35]:
#Limpiaremos las columnas de texto y para hacemo vamos a definir las stopwords

stopwords_english = set(stopwords.words('english'))
stopwords_spanish = set(stopwords.words('spanish'))

In [36]:
# Función para limpiar tokenizar nuestras columnas
def clean_text(text):
    # Tokenización
    tokens = word_tokenize(text)
    # Eliminar signos de puntuación y convertir a minúsculas
    tokens = [word.lower() for word in tokens if word.isalpha()]
    # Filtrar stopwords en inglés y español
    tokens = [word for word in tokens if word not in stopwords_english and word not in stopwords_spanish]
    return tokens



In [37]:
df_recomendacion['clean_title'] = df_recomendacion['title'].apply(lambda x: clean_text(x))
df_recomendacion['clean_overview'] = df_recomendacion['overview'].dropna().apply(lambda x: clean_text(x))

In [38]:
#Eliminamos las columna title y overview
df_recomendacion = df_recomendacion.drop(['overview'],axis=1)
# df_recomendacion.head()


In [39]:
# Convertimos las columnas seleccionadas para el modelo a str
df_recomendacion['clean_title'] = df_recomendacion['clean_title'].astype(str)
df_recomendacion['clean_overview'] = df_recomendacion['clean_overview'].astype(str)
df_recomendacion['genresname'] = df_recomendacion['genresname'].astype(str)

In [41]:
# Combinamos las columnas seleccionadas en una sola columna, dandole mas peso al titulo y al genero
df_recomendacion['combined_features'] = df_recomendacion['clean_title']+ ' '+ df_recomendacion['clean_title'] + ' ' + df_recomendacion['clean_overview'] + ' ' + df_recomendacion['genresname']+ ' ' + df_recomendacion['genresname']+ ' ' + df_recomendacion['genresname']


In [42]:
# Combinar las stop words en un solo conjunto
combinacion_stop_words = list(set(stopwords_english).union(set(stopwords_spanish)))

In [43]:
# Resetear los índices
df_recomendacion.reset_index(drop=True, inplace=True)

In [44]:
# Vectorización utilizando TF-IDF con stop words en inglés y español
vectorizer = TfidfVectorizer(stop_words=combinacion_stop_words, max_features=5000)
tfidf_matrix = vectorizer.fit_transform(df_recomendacion['combined_features'])
# Verificar la forma de la matriz TF-IDF resultante
print("Forma de la matriz TF-IDF antes de TruncatedSVD:", tfidf_matrix.shape)

# Aplicar TruncatedSVD para reducir la dimensionalidad
svd = TruncatedSVD(n_components=100)
tfidf_matrix_svd = svd.fit_transform(tfidf_matrix)

# Verificar la forma de la matriz después de TruncatedSVD
print("Forma de la matriz después de TruncatedSVD:", tfidf_matrix_svd.shape)

Forma de la matriz TF-IDF antes de TruncatedSVD: (36034, 5000)
Forma de la matriz después de TruncatedSVD: (36034, 100)


In [45]:
# Calcular la similitud del coseno
cosine_sim = cosine_similarity(tfidf_matrix_svd, tfidf_matrix_svd) 


In [49]:
# Definir la función get_recommendations
def get_recommendations(title, df=df_recomendacion, cosine_sim=cosine_sim):
    try:
        # Obtener el índice de la película que coincide con el título
        idx = df[df['title'].str.lower() == title.lower()].index[0]

        # Obtener los scores de similitud de todas las películas con esa película
        sim_scores = list(enumerate(cosine_sim[idx]))

        # Ordenar las películas según la puntuación de similitud
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # Obtener las 5 películas más similares (excepto la misma película)
        sim_scores = sim_scores[1:6]

        # Obtener los índices de las películas más similares
        movie_indices = [i[0] for i in sim_scores]

        # Retornar los títulos de las películas más similares
        return df['title'].iloc[movie_indices].tolist()
    except IndexError:
        raise HTTPException(status_code=404, detail=f"No se encontraron películas similares para '{title}'. Asegúrate de haber puesto correctamente el título.")


In [50]:
ejemplo = get_recommendations('toy story')
ejemplo

['Toy Story 2', 'Botsman i Popugay', 'Banana', 'Lorenzo', 'Feed the Kitty']